# Check contents of MongoDB

Install required libraries.

In [ ]:
import os
import numpy as np
import pandas as pd
import krippendorff

from pymongo import MongoClient
from dotenv import load_dotenv
from scipy.stats import mode
from pingouin import intraclass_corr

In [2]:
load_dotenv() # load gitignore

mongo_user = os.getenv('MONGO_USER')
mongo_password = os.getenv('MONGO_PASSWORD')
cluster_url = os.getenv('MONGO_CLUSTER_URL')
gradio_user = os.getenv('GRADIO_USER')
gradio_password = os.getenv('GRADIO_PASSWORD')

In [3]:
connection_url = f"mongodb+srv://{mongo_user}:{mongo_password}@{cluster_url}"
client = MongoClient(connection_url)
db = client["thesis"]
collection = db["labeling"]

Get all raters with their amount of data annotated.

In [4]:
def get_labels_per_annotator():
    """Get the number of labels given by each annotator."""
    pipeline = [
        {"$unwind": "$ratings"},
        {"$group": {
            "_id": "$ratings.rater",  # group by annotator name
            "count": {"$sum": 1}  # count number of ratings per annotator
        }},
        {"$sort": {"count": -1}}  # sort in descending order
    ]

    results = list(collection.aggregate(pipeline))
    return {result["_id"]: result["count"] for result in results}

In [5]:
labels_per_annotator = get_labels_per_annotator()
print("Names & labels per annotator:", labels_per_annotator)

Names & labels per annotator: {'ej': 346, 'basil': 346, 'renz': 346, 'pia': 346, 'Thomas': 231, 'sona': 165, 'gaurav': 5, 'michal': 2, 'Alex T': 2, 'Tuna ': 1, 'holding three fingers up': 1, 'kiko': 1, 'character intro': 1, 'H': 1, 'tim': 1, 'Abhi': 1, 'kaku': 1, 'Ghost ': 1}


Delete specified annotator.

In [6]:
rater_to_delete = "Pia"

result = collection.update_many(
    {"ratings.rater": rater_to_delete},
    {"$pull": {"ratings": {"rater": rater_to_delete}}}
)

print(f"Number of deleted entries: {result.modified_count}")

Number of deleted entries: 128


Initial analysis of entries for a brief overview.

In [11]:
annotators = {"sona", "Thomas", "ej", "renz", "basil"}

pipeline = [
    {"$match": {"ratings.rater": {"$in": list(annotators)}}},
    {"$unwind": "$ratings"},
    {"$match": {"ratings.rater": {"$in": list(annotators)}}},
    {"$project": {"rater": "$ratings.rater", "alignment": "$ratings.alignment", "quality": "$ratings.quality", "consistency": "$ratings.consistency", "overall": "$ratings.overall"}}
]

data = list(collection.aggregate(pipeline))
df = pd.DataFrame(data)

In [45]:
df.head()

,_id,rater,alignment,quality,consistency,overall
0,677d856e8640497613a5ea32,Thomas,1,5,6,1
1,677d856e8640497613a5ea32,renz,1,10,10,1
2,677d856e8640497613a5ea32,ej,1,10,10,2
3,677d856e8640497613a5ea32,basil,0,10,10,0
4,677d856e8640497613a5ea35,Thomas,7,5,5,6


In [43]:
aspects = ["alignment", "quality", "consistency", "overall"]
stats = {}
icc_values = []
alpha_values = []

for aspect in aspects:
    mean_val = df[aspect].mean()
    median_val = df[aspect].median()
    mode_result = mode(df[aspect], nan_policy='omit')
    mode_val = np.atleast_1d(mode_result.mode)[0] if mode_result.count.size > 0 else np.nan
    std_dev = df[aspect].std()

    # group by annotator
    grouped = df.groupby('rater')[aspect].apply(list)

    # convert to df
    icc_data = pd.DataFrame(grouped.tolist()).T  # transpose -> each column is a rater
    icc_data.columns = grouped.index

    # column naming
    icc_data = icc_data.reset_index().melt(id_vars=['index'], var_name='raters', value_name='ratings')
    icc_data.rename(columns={'index': 'targets'}, inplace=True)

    # ICC
    icc_results = intraclass_corr(data=icc_data.dropna(), targets='targets', raters='raters', ratings='ratings', nan_policy='omit').round(3)
    icc_value = icc_results[icc_results['Type'] == 'ICC2']['ICC'].values[0] if not icc_results.empty else np.nan
    icc_values.append(icc_value)

    stats[aspect] = {
        "Mean": mean_val,
        "Median": median_val,
        "Mode": mode_val,
        "Standard Deviation": std_dev,
        "ICC": icc_value
    }

avg_icc = np.nanmean(icc_values)

print("ratings per aspect")
for aspect, values in stats.items():
    print(f"\n{aspect}")
    for key, val in values.items():
        print(f"{key}: {val:.3f}")

print(f"\nOverall Average ICC: {avg_icc:.3f}")

ratings per aspect

alignment
Mean: 8.071
Median: 10.000
Mode: 10.000
Standard Deviation: 3.038
ICC: 0.280

quality
Mean: 7.502
Median: 8.000
Mode: 10.000
Standard Deviation: 2.639
ICC: 0.130

consistency
Mean: 8.206
Median: 9.000
Mode: 10.000
Standard Deviation: 2.416
ICC: 0.100

overall
Mean: 7.173
Median: 8.000
Mode: 10.000
Standard Deviation: 2.897
ICC: 0.224

Overall Average ICC: 0.183
